# Breweries, Beer, and Breviews: Unraveling the Global Beer Preferences
## Introduction

## Repository structure




## Include Libraries and initial settings
### Import all the libraries

In [1]:
# Import all the libraries
import pandas as pd
import plotly.express as px
from src.utils.plots import *
import numpy as np
import bar_chart_race as bcr
import networkx as nx
from pyvis.network import Network
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression

# Shutdown FutureWarning and UserWarning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

# Define the folder
FOLDER = 'data/processed/'
SAVING_FOLDER = 'docs/plots/'

### Load and filter the data
#### Load the data

In [2]:
df_beers = pd.read_parquet(FOLDER + 'beers.pq')
df_breweries = pd.read_parquet(FOLDER + 'breweries.pq')
df_users = pd.read_parquet(FOLDER + 'users.pq')
df_ratings_no_text = pd.read_parquet(FOLDER + 'ratings_no_text.pq')

## Data presentation
This section provides an overview of the dataset, which has been carefully cleaned and is nearly ready for analysis. We will summarize the data, display the first few rows, and describe the data types of the columns. For columns that are not self-explanatory, a brief description is included. <br><br>
Let's begin our analysis with the beers.

In [3]:
df_beers.head(5)

,beer_id,beer_name,brewery_id,brewery_name,style,abv
3,155699,Ards Bally Black Stout,13538,Ards Brewing Co.,Stout,4.6
4,239097,Ards Belfast 366,13538,Ards Brewing Co.,Golden Ale/Blond Ale,4.2
5,380930,Ards Bittles Christmas,13538,Ards Brewing Co.,Bitter,4.2
6,234108,Ards Bittles Citra Autumn Ale,13538,Ards Brewing Co.,American Pale Ale,5.1
7,396830,Ards Cardy Man,13538,Ards Brewing Co.,Stout,4.2


In [4]:
format_numeric_values = lambda x: f"{x:,.0f}"
print(f"In the platform there are {format_numeric_values(df_beers.shape[0])} different beers")

In the platform there are 314,242 different beers


Now let's take a look at the breweries.

In [5]:
df_breweries.head(5)

,brewery_id,brewery_name,country_brewery,state_brewery
1,13538,Ards Brewing Co.,United Kingdom,None
2,22304,Barrahooley Craft Brewery,United Kingdom,None
3,22818,Boundary,United Kingdom,None
4,24297,Brewbot Belfast,United Kingdom,None
5,27006,Bullhouse Brewing Company,United Kingdom,None


We have chosen, to ease the analysis processes, to split the location into two columns:
- One for the country.
- A separate column for US states.

This split was specifically applied to the US because it has a large number of users and breweries and combining all states into a single "US" category would have obscured important insights. Additionally, the size and population of many US states are comparable to those of entire countries. This approach was not extended to other countries due to the lack of state-level data for them. <br><br>
The same choice has been made for all the location columns.

In [6]:
print(f"In the platform there are {format_numeric_values(df_breweries.shape[0])} different breweries")

In the platform there are 19,670 different breweries


Let's now add the location columns to the beers dataset

In [7]:
df_beers = df_beers.join(df_breweries[['brewery_id', 'country_brewery', 'state_brewery']].set_index('brewery_id'), on='brewery_id').rename(columns={'country_brewery': 'country_beer', 'state_brewery': 'state_beer'})
df_beers.head(5)

,beer_id,beer_name,brewery_id,brewery_name,style,abv,country_beer,state_beer
3,155699,Ards Bally Black Stout,13538,Ards Brewing Co.,Stout,4.6,United Kingdom,None
4,239097,Ards Belfast 366,13538,Ards Brewing Co.,Golden Ale/Blond Ale,4.2,United Kingdom,None
5,380930,Ards Bittles Christmas,13538,Ards Brewing Co.,Bitter,4.2,United Kingdom,None
6,234108,Ards Bittles Citra Autumn Ale,13538,Ards Brewing Co.,American Pale Ale,5.1,United Kingdom,None
7,396830,Ards Cardy Man,13538,Ards Brewing Co.,Stout,4.2,United Kingdom,None


Let's go back to analyze our data with the users dataset now.

In [8]:
df_users.head(5)

,user_id,user_name,joined,country_user,state_user
0,175852,Manslow,2012-05-20 10:00:00,Poland,None
2,288889,Sibarh,2013-11-16 11:00:00,Poland,None
3,250510,fombe89,2013-03-22 11:00:00,Spain,None
4,122778,kevnic2008,2011-02-02 11:00:00,Germany,None
5,227834,Beerhunter111,2012-11-12 11:00:00,Germany,None


In [9]:
print(f"In the platform there are {format_numeric_values(df_users.shape[0])} different users")

In the platform there are 46,993 different users


And finally let's have a look at the ratings dataset.

In [10]:
df_ratings_no_text.head(5)

,date,beer_id,user_id,brewery_id,abv,style,rating,mouthfeel,taste,appearance,aroma,overall,year,brewery_name,country_brewery,state_brewery,country_user,state_user
38,2015-04-16 12:00:00,155699,151109,13538,4.6,Stout,3.7,2.0,8.0,3.0,7.0,17.0,2015,Ards Brewing Co.,United Kingdom,None,Ireland,None
41,2013-10-01 12:00:00,155699,224156,13538,4.6,Stout,4.0,3.0,5.0,5.0,8.0,19.0,2013,Ards Brewing Co.,United Kingdom,None,United Kingdom,None
42,2013-06-18 12:00:00,155699,88501,13538,4.6,Stout,3.6,4.0,7.0,4.0,7.0,14.0,2013,Ards Brewing Co.,United Kingdom,None,New Zealand,None
43,2011-11-17 12:00:00,155699,128086,13538,4.6,Stout,4.0,5.0,7.0,5.0,7.0,16.0,2011,Ards Brewing Co.,United Kingdom,None,United Kingdom,None
44,2013-11-23 12:00:00,239097,128086,13538,4.2,Golden Ale/Blond Ale,3.0,3.0,5.0,4.0,6.0,12.0,2013,Ards Brewing Co.,United Kingdom,None,United Kingdom,None


In [11]:
print(f"In the platform there are {format_numeric_values(df_ratings_no_text.shape[0])} different reviews")

In the platform there are 6,424,384 different reviews


The ratings comes from the [RateBeer](ratebeer.com) website which has the following rating system (source [RateBeer Scores](https://www.ratebeer.com/our-scores)):
- <b>Aroma</b>: The smell of the beer
- <b>Appearance</b>: The color, clarity, head and visual carbonation of this beer.
- <b>Taste</b>: The flavors in this beer, thinking about the palate, bitterness and finish.
- <b>Mouthfeel</b>: The body of the beer, carbonation and astringency.
- <b>Overall</b>: The overall characteristics and your personal experience of the beer.

Appearance and Mouthfeel are each scored out of 5. Aroma and Taste are scored out of 10. While Overall is scored out of 20. These all combine to give the beer a total score out of 50 (the <b>Rating</b> column), which is then divided and displayed as a score out of 5 for each rating. <br>
The fact that the <b>Rating</b> column is the sum of the other columns is important to keep in mind when analyzing the data in particular when doing correlation analysis. No normalization has been performed at this stage.<br><br>
The country of the brewery is computed from the <code>brewery</code> dataset while the country of the user is computed from the <code>user</code> dataset.

In [12]:
print(f"In the platform there are {format_numeric_values(df_ratings_no_text.shape[0])} different ratings")
print(f"The first rating was made on {df_ratings_no_text['date'].min()}")
print(f"The last rating was made on {df_ratings_no_text['date'].max()}")

In the platform there are 6,424,384 different ratings
The first rating was made on 2002-01-01 12:00:00
The last rating was made on 2016-12-31 12:00:00


We see that our dataset ends and begin at the beginning of January 2002 and ends at the end of December 2016 with a total of 6,424,384 different ratings. Some of the ratings have been removed to ensure consistent results. <br>
We indeed observed that by considering earlier and later ratings the results for those years were not consistent or reasonable mainly because:
- The first year that where we have data from the beginning of January is 2001 but the ratings on this year are significantly lower than the rest and doesn't follow the trend of the other years.
- The last year that ends at the end of December is 2016 because on 2017 we have not data for the second half of the year. The number of ratings in this year indeed are completely off if compared with the other years.

Now that we had an overview of the data let's have a look at the geographical distribution of our data in the world.

In [13]:
# Include the class that contains all the code useful for the data presentation
from src.processing import presentation as pr
presentation = pr.DataPresentation(df_beers, df_breweries, df_users, df_ratings_no_text, f'{SAVING_FOLDER}/data_presentation')

# Get the aggregated spatial data
df_no_US, df_US = presentation.get_spatial_data()

# Define some options for the plot
options = {
    "plots": [
        { 'label': 'Beers per country', 'location_label': 'location', 'z_label': 'count', 'colorscale': 'Blues', 'show_title': False },
        { 'label': 'Users per country', 'location_label': 'location', 'z_label': 'count', 'colorscale': 'Blues' , 'show_title': False},
        { 'label': 'Breweries per country', 'location_label': 'location', 'z_label': 'count', 'colorscale': 'Blues' , 'show_title': False},
        { 'label': 'Number of ratings based on the brewery country', 'location_label': 'location', 'z_label': 'count', 'colorscale': 'Blues' , 'show_title': False},
        { 'label': 'Number of ratings based on the reviewer country', 'location_label': 'location', 'z_label': 'count', 'colorscale': 'Blues' , 'show_title': False},
    ]
}

# Plot the map
plot_map(df_no_US, df_US, options, SAVING_FOLDER + 'data_presentation', 'ratings_map')

In [14]:
presentation.plot_top_5()

We see that the data are spread in all the world, with a higher concentration in the United States, if considered as a whole. If we consider the single states of the United States then we have that Enland, Germany, Belgium and Canada are very active together with California that is the most active state in the United States. We can say that while the dataset is more biased towards united states we'll show in the further analysis that by considering the single states and not only the entire country we can highlight some interesting differences within the United States.

And finally let's review the temporal distribution of the data.

In [15]:
ratings_temporal_grouping, users_temporal_grouping = presentation.get_temporal_data()
presentation.plot_temporal(ratings_temporal_grouping, 'Year', 'Number of ratings', 'Number of ratings per year')

In [16]:
presentation.plot_temporal(users_temporal_grouping, 'Year', 'Number of users', 'Number of users per year')

We see that the number of ratings peak in 2015 while the number of new users peak in 2012 with a decline after that year.

## Global ratings analysis
In this section we are going to analyze the preferences of the users of the platform in a general way.
### General study of the distribution of the ratings
First of all we are going to study how the grades are distributed in the different categories. This will help us understand if there are specific bias towards some categories and whether the users are more or less demanding in some categories.

In [17]:
# Include all the code needed for the global ratings analysis
from src.processing import global_pres as gp
global_pres = gp.Global(df_beers, df_breweries, df_users, df_ratings_no_text, save_folder=SAVING_FOLDER)

# Display the ratings distribution
global_pres.display_ratings_distribution()

Before doing any conclusion also let's compute some key metrics for each category.

In [18]:
global_pres.display_ratings_statistics()

Statistics of the ratings 
 
 
 Rating 
 Mean 
 Std 
 Median 
 Skewness 
 Kurtosis 
 Can reject H0 (95%) 
 
 
 
 
 mouthfeel 
 3.27 
 0.79 
 3.0 
 -0.19 
 0.25 
 True 
 
 
 taste 
 6.48 
 1.53 
 7.0 
 -0.79 
 1.01 
 True 
 
 
 appearance 
 3.43 
 0.77 
 3.0 
 -0.15 
 0.26 
 True 
 
 
 aroma 
 6.4 
 1.53 
 7.0 
 -0.77 
 1.0 
 True 
 
 
 overall 
 13.18 
 3.14 
 14.0 
 -0.99 
 1.4 
 True 
 
 
 rating 
 3.28 
 0.68 
 3.4 
 -0.92 
 1.42 
 True

We observe multiple interesting aspects:
- **Central Tendency Bias:**  
  All the ratings have a mean and median close to the center of the rating scale. This suggests users exhibit a central tendency bias, where they avoid extreme judgments and rate items near the middle of the scale. However, the Shapiro-Wilk test at 95% confidence indicates the data are not normally distributed, as also evident from the distribution plots.  

- **Category-specific Observations:**  
  While the mean and median are generally close to the center of the rating scale, there are notable differences, both numerically and visually:  

  - **Mouthfeel and Appearance:**  
    - The mean is slightly higher than the median, and skewness is lower compared to other categories.  
    - This suggests users are more demanding in these categories, leading to generally lower ratings.  
    - The subjectivity and difficulty in evaluating these attributes could also play a role. For instance, what constitutes a "5-star" mouthfeel is often ambiguous.  

  - **Taste and Aroma:**  
    - The median is slightly higher than the mean, indicating users favor these categories and tend to give them higher ratings.  
    - This suggests users genuinely enjoy these aspects, resulting in more favorable evaluations.  

  - **Overall Rating:**  
    - The overall score leans toward the higher end of the scale, potentially influenced by taste and aroma ratings.  
    - This warrants further investigation in future analyses.  

### Correlation between the ratings attributes
Here we are gonna analyze the correlation between the ratings attributes given by the users. In this section we won't consider the rating but rather the overall score as an overall rate of the experience. This is reasonable since:
- The overall is used to evaluate in general the experience of the beer
- The rating is computed from the other parameters so studing the correlation of the parameters with respect to it has little to no significance.

In [19]:
global_pres.plot_correlation_matrix(df_ratings_no_text[['mouthfeel','taste','appearance','aroma','overall','abv']], 'docs/plots/ratings_distribution/correlation_matrix.html')

We see that taste and aroma show the highest correlations with overall beer ratings, at 0.86 and 0.77, respectively. These attributes dominate the beer experience given their impact on the overall flavor profile.

Appearance and palate, on the other hand, have moderate correlations with overall beer ratings, at 0.50 and 0.66, respectively. The lower correlation, compared with the one of taste and aroma, suggests that these attributes play a secondary role in the overall experience, either for the complexity of the evaluation or for the less significance in the overall experience.

Alcohol by volume (ABV) has the lowest correlation overall (with a correlation of 0.37 with the overall ratings) even if it's still significant. This suggests that there is some link between the ABV and the users preferences that we are going to explore further in the next sections.

We see that there is a strong relation between the different parameters and the overall score. Can we use them to predict wether or not the user will enjoy the beer?

In [20]:
results, r_squared, rmse = global_pres.regression_analysis(df_ratings_no_text)
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                overall   R-squared:                       0.773
Model:                            OLS   Adj. R-squared:                  0.773
Method:                 Least Squares   F-statistic:                 4.379e+06
Date:                Mon, 16 Dec 2024   Prob (F-statistic):               0.00
Time:                        14:34:20   Log-Likelihood:            -9.3563e+06
No. Observations:             5139954   AIC:                         1.871e+07
Df Residuals:                 5139949   BIC:                         1.871e+07
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         13.1768      0.001      2e+04      0.0

In [ ]:
print(f"The R-squared value at test is {r_squared}")
print(f"The RMSE value at test is {rmse}")

The R-squared value at test is 0.7731865103961637
The RMSE value at test is 1.495436930543424


It seems we can do this!</br><br>
From the linear regression model we can clearly see that the four parameters are significant in predicting the overall experience of an user and whether or not one will like the beer. We see that approximately 77% of the variance is explained by the model with a test set and that the RMSE is quite low, approximately 1.50, which is not significant in a scale with a granularity of 1. <br>
From the p value we see that all the parameters are significant in the prediction of the overall grade (the p value for all of them is almost zero). <br>
We see that taste has almost double the impact on the overall experience than aroma, with the first one that has a coefficient of <b>1.7843</b> vs a coefficient of <b>0.7184</b> for the second parameter. We see that the appearance and the mouthfeel have a much lower impact than the others (respectively <b>0.4159</b> and <b>0.1296</b>). <br><br>
From these results we have a significant insight: if user does't like the taste of a beer than they'll likely won't like the beer at all.

### Rating in the different countries
Here we are going to see which are the favourite beer locations in the world. We are going to consider the rating here to have a metric that summarize all the other parameters, given that now we know how they influence the overall experience and in general how impactful each of them on user preferences.

In [22]:
# Compute the average rating per country
average_rating_no_US = df_ratings_no_text[df_ratings_no_text['country_brewery'] != 'United States'].groupby('country_brewery')['rating'].mean().reset_index().rename(columns={'country_brewery':'location', 'rating':'count'})
average_rating_US = df_ratings_no_text[df_ratings_no_text['country_brewery'] == 'United States'].groupby('state_brewery')['rating'].mean().reset_index().rename(columns={'state_brewery':'location', 'rating':'count'})

# Plot everything
options = {
    "title": "Average Ratings by Country and US State",
    "plots": [{
        'label': 'Beers per country',
        'location_label': 'location',
        'z_label': 'count',
        'colorscale': 'Blues'
    }]
}
plot_map(average_rating_no_US, average_rating_US, options, SAVING_FOLDER + 'ratings_distribution', 'world_ratings')

In [23]:
average_rating = pd.concat([average_rating_no_US, average_rating_US]).sort_values('count', ascending=False)
average_rating.head(10)

,location,count
1,Alaska,3.629238
35,Oklahoma,3.618523
4,California,3.601744
34,Ohio,3.551829
12,Illinois,3.546681
36,Oregon,3.532074
47,Washington DC,3.512826
8,Belgium,3.498091
28,New Hampshire,3.488963
9,Georgia,3.487447


In [24]:
average_rating = pd.concat([average_rating_no_US, average_rating_US]).sort_values('count', ascending=True)
average_rating.head(10)

,location,count
81,Nicaragua,1.787751
41,Guatemala,1.818750
29,El Salvador,1.823953
22,Cuba,1.908679
28,Egypt,1.913043
113,Venezuela,1.921269
109,Uganda,1.922050
76,Myanmar,1.945098
104,Tanzania,1.976498
26,Dominican Republic,1.978745


### Rating evolution over time in the different countries

In [25]:
df_state_no_US, df_state_US = global_pres.get_spatial_data_ratings_evolving(df_ratings_no_text)

# Define the options for the plot
options = {
    'time_range': range(df_ratings_no_text['date'].dt.year.min(), df_ratings_no_text['date'].dt.year.max() + 1),
    'time_label': 'year',
    'location_label': 'location',
    'value_label': 'count',
    'range_color': [1, 4],
    'color_scale': 'Viridis'
}

# Display the plot
plot_map_time(df_state_no_US, df_state_US, options, SAVING_FOLDER + '/ratings_distribution/average_rating_map_time.html')

In [26]:
global_pres.plot_median_mean_evolution_time_ratings(df_ratings_no_text,SAVING_FOLDER+'/ratings_distribution')

## ABV and style analysis
#### ABV analysis
In this section we'll continue by analyzing the popularity of ABV and style in the world and we'll also study how these have evolved over time.

In [ ]:
# Define some useful variables
MAX_ABV = 20
NUMBER_OF_SAMPLES_ABV = 201
MIN_NUMBER_OF_RATING = 250

# Process the data
beer_ratings = []
linspace = np.linspace(0, MAX_ABV, NUMBER_OF_SAMPLES_ABV)
for year in sorted(df_ratings_no_text['date'].dt.year.unique()):
    # Filter the data by the year
    df_year = df_ratings_no_text[df_ratings_no_text['year'] == year]
    
    # Iterate within the ABV range
    for i in range(len(linspace) - 1):
        # Filter the data
        min_abv = round(linspace[i], 2)
        max_abv = round(linspace[i + 1], 2)

        # Compute the matrics
        filtered = df_year[(df_year['abv'] >= min_abv) & (df_year['abv'] < max_abv)]
        ratings = filtered['rating'].mean()
        nbr_ratings = filtered['rating'].count()

        # Append the data
        if nbr_ratings > MIN_NUMBER_OF_RATING:
            beer_ratings.append({'year': year, 'abv': (min_abv+max_abv)/2, 'rating': ratings, 'nbr_ratings': nbr_ratings})

# Convert to DataFrame
beer_ratings = pd.DataFrame(beer_ratings)

# Do the plot
fig = px.scatter(beer_ratings, x='abv', y='rating', size='nbr_ratings', hover_name='abv',animation_frame='year', labels={'abv': 'ABV:', 'rating': 'Rating:', 'nbr_ratings': 'Number of ratings:'},range_x=[0, 20], range_y=[2.25, 4.75])
fig.update_traces(marker=dict(line=dict(width=1, color='DarkSlateGrey')))
fig.update_layout(showlegend=False)
fig.update_xaxes(title_text='ABV')
fig.update_yaxes(title_text='Rating')
fig.show()

In [ ]:
# Compute the metrics
avg_abv = df_ratings_no_text.groupby(df_ratings_no_text['date'].dt.year).agg({'abv': 'mean'}).reset_index().rename(columns={'date': 'year'})
corr = avg_abv['year'].corr(avg_abv['abv'], method='spearman')

# Plot the data
plots_values_over_time({'average_abv': avg_abv}, 'year', 'abv', 'Year', 'Average ABV Over Years', 'Average ABV')

In [ ]:
# Create the variables
row_no_US = []
row_US = []

# Iterate over the years
for year in df_ratings_no_text['date'].dt.year.unique():
    df_state_US = df_ratings_no_text[(df_ratings_no_text['date'].dt.year == year) & (df_ratings_no_text['country_brewery'] == 'United States')].groupby('state_brewery').agg({'abv': 'mean'}).reset_index()
    row_US += [{'year': year, 'location': state, 'avg_abv': abv} for state, abv in zip(df_state_US['state_brewery'], df_state_US['abv'])]

    df_state_no_US = df_ratings_no_text[(df_ratings_no_text['date'].dt.year == year) & (df_ratings_no_text['country_brewery'] != 'United States')].groupby('country_brewery').agg({'abv': 'mean'}).reset_index()
    row_no_US += [{'year': year, 'location': state, 'avg_abv': abv} for state, abv in zip(df_state_no_US['country_brewery'], df_state_no_US['abv'])]
df_states_no_US = pd.DataFrame(row_no_US)
df_states_US = pd.DataFrame(row_US)

# Filter the data
nbr_years_considered = df_states_no_US.groupby('location').agg({'year': 'count'}).reset_index()
nbr_years_considered = nbr_years_considered[nbr_years_considered['year'] == nbr_years_considered['year'].max()]
df_states_no_US = df_states_no_US[df_states_no_US['location'].isin(nbr_years_considered['location'])]

# Filter the data
nbr_years_considered = df_states_US.groupby('location').agg({'year': 'count'}).reset_index()
nbr_years_considered = nbr_years_considered[nbr_years_considered['year'] == nbr_years_considered['year'].max()]
df_states_US = df_states_US[df_states_US['location'].isin(nbr_years_considered['location'])]

# Define the options
options = {
    'title': 'Average ABV by State Over the Years',
    'time_range': range(df_ratings_no_text['date'].dt.year.min(), df_ratings_no_text['date'].dt.year.max() + 1),
    'time_label': 'year',
    'location_label': 'location',
    'value_label': 'avg_abv',
    'range_color': [4, 8],
    'color_scale': 'Viridis'
}
# Display the plot
plot_map_time(df_states_no_US, df_states_US, options)

#### Style analysis
Here we are going to do our analysis on the style

In [ ]:
# Choose the number of elements to display
number_of_elements_displayed = 19

# Do the styling computations
styles_counted = df_ratings_no_text['style'].value_counts().reset_index()
top_styles = styles_counted.head(number_of_elements_displayed).copy()
top_styles.loc[len(top_styles)] = {'style': 'Other', 'count': styles_counted['count'][number_of_elements_displayed:].sum()}

# Display a pie chart with gradient colors
fig = px.pie(top_styles, values='count', names='style', title='Distribution of the styles of beers', color_discrete_sequence=px.colors.sequential.Viridis, hole=0.6)
fig.update_traces(textinfo='label+percent', textfont_size=14)
fig.show()

In [ ]:
# Find the countries style preferences
countries_style_preferences = {}
unique_styles = set()
for country in countries_min_number_reviews_users:
    # Filter the data
    df_country = df_ratings_no_text[df_ratings_no_text['country_user'] == country]
    
    # Compute the style preferences
    style_preferences = df_country['style'].value_counts().reset_index().head(1).iloc[0]['style']

    # Add into countries_style_preferences the data
    countries_style_preferences[country] = style_preferences
    unique_styles.add(style_preferences)

In [ ]:
# Create a network graph
G = nx.Graph()

# Add a node for each country
for style in unique_styles:
    G.add_node(style)

# Add a node for each country and connect it to the style
for country, style in countries_style_preferences.items():
    G.add_node(country)
    G.add_edge(country, style, weight=1)

# Plot the network with plot
net = Network(height='750px', width='100%', notebook=True)
net.from_nx(G)
net.repulsion(node_distance=150)  # Adjust the node distance for closer visualization
net.show('docs/plots/beer_styles_networl.html');

In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Prepare data for the chord diagram
countries = list(countries_style_preferences.keys())
styles = list(unique_styles)

# Combine countries and styles for all_nodes
all_nodes = countries + styles
node_indices = {node: idx for idx, node in enumerate(all_nodes)}

# Create the source and target lists for the chord diagram
sources = []
targets = []
values = []

for country, style in countries_style_preferences.items():
    sources.append(node_indices[country])
    targets.append(node_indices[style])
    values.append(1)  # Set weight for each connection

# Create a color palette for nodes
num_countries = len(countries)
num_styles = len(styles)
colors = [f"rgba(0, 128, 255, 0.8)" for _ in range(num_countries)] + [f"rgba(255, 128, 0, 0.8)" for _ in range(num_styles)]

# Assign unique colors to each cluster of connections
link_colors = []
style_color_map = {style: f"rgba({max(0, 255 - i * 30)}, {min(255, 100 + i * 30)}, {min(255, 150 + i * 20)}, 0.6)" for i, style in enumerate(styles)}
for source, target in zip(sources, targets):
    link_colors.append(style_color_map[all_nodes[target]])

# Create the chord diagram using Plotly
fig = go.Figure(
    data=[
        go.Sankey(
            node=dict(
                pad=20,
                thickness=30,
                line=dict(color="black", width=1),
                label=all_nodes,
                color=colors,
            ),
            link=dict(
                source=sources,  # Indices of source nodes
                target=targets,  # Indices of target nodes
                value=values,    # Values for connections
                color=link_colors,
                hovertemplate='%{source.label} → %{target.label}<extra></extra>',
            ),
        )
    ]
)

# Update layout for better visualization
fig.update_layout(
    title_text="Countries and Their Preferred Beer Styles",
    font_size=12,
    title_font_size=18,
    title_font_color="darkblue",
    title_x=0.5,
    height=900,
    width=1000,
    plot_bgcolor="rgba(240, 240, 240, 0.9)",
)

# Save the plot to an HTML file
fig.write_html("docs/plots/beer_styles_chord_diagram.html")

# Show the plot
fig.show()


In [ ]:
# Could be interesting to show the evolution of the style preferences over the years with the network graph

In [ ]:
# Prepare the data
all_styles = df_ratings_no_text['style'].unique()

# Create a dataframe with the top_10_styles_list elements as columns
row = []
for year in sorted(df_ratings_filtered['date'].dt.year.unique()):
    # Get the data for the year
    df_year = df_ratings_filtered[df_ratings_filtered['date'].dt.year == year]

    # Compute the style preferences
    style_preferences = df_year['style'].value_counts().reset_index().head(10)

    # Compute the ABV for the top 10 styles
    for style in style_preferences['style'].values:
        # Get the data
        df_style = df_year[df_year['style'] == style]

        # Compute the average ABV
        avg_abv = df_style['abv'].mean()

        # Append the data
        row.append({'year': year, 'style': style, 'avg_abv': avg_abv, 'count': style_preferences[style_preferences['style'] == style]['count'].values[0]})

# Create the dataframe
df_style_avg_abv = pd.DataFrame(row)

In [ ]:
# Create the plot with the slider for the years
fig = px.bar(df_style_avg_abv, x='style', y='count', animation_frame='year', title='Top 10 styles over the years', range_y=[0, 100000], color='avg_abv', range_color=[0, 10], color_continuous_scale='Blues')
# Use log scale for the y-axis
fig.update_xaxes(title_text='Style')
fig.update_yaxes(title_text='Number of ratings')
fig.show()

In [ ]:
# Prepare the data
all_styles = df_ratings_no_text['style'].unique()

# Create a dataframe with the top_10_styles_list elements as columns
nbr_ratings_per_style = pd.DataFrame(columns=['year'] + all_styles)
for year in sorted(df_ratings_filtered['date'].dt.year.unique()):
    # Get the data for the year
    df_year = df_ratings_filtered[df_ratings_filtered['date'].dt.year <= year]

    # Compute the number of ratings per style
    styles_counted = df_year['style'].value_counts().reset_index()

    # Create the dictionary
    row = {style: count for style, count in zip(styles_counted['style'], styles_counted['count'])}
    row['year'] = year

    # Add the row to the dataframe
    nbr_ratings_per_style = pd.concat([nbr_ratings_per_style, pd.DataFrame([row])])

# Fill the missing values and set the index
nbr_ratings_per_style = nbr_ratings_per_style.fillna(0).set_index('year')

# Set to integer the types
nbr_ratings_per_style = nbr_ratings_per_style.astype(int)

In [ ]:
# Display race bar chart
bcr.bar_chart_race(nbr_rating_per_style, period_length=1000, title='Total number of ratings per beer style', n_bars=10, steps_per_period=50, figsize=(8, 6), cmap='tab20', period_fmt='Year: {x}')

## Brewery popularity analysis
In this section we are going to see whether user likes a lot beers from specific breweries from specific countries and in general we'll focus on understanding the impact of breweries on the popularity of beers.